# Development scratchbook

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from etfs.security.io import read_yahoo_csv, retrieve_yahoo_quote
from etfs.stats.basics import resample_df, runrate_column, shift_column, return_column, difference, rsq
from etfs.security import security
from etfs.utils.helpers import todays_date

## Retrieve quotes from Yahoo! Finance

In [ ]:
df = retrieve_yahoo_quote(ticker='VO', startdate='19000101', enddate='20180204')

In [ ]:
df.head()

## Alternatively: read in files

In [ ]:
filepath = '../data/VNQ.csv'

In [ ]:
df = read_yahoo_csv(path=filepath)

In [ ]:
df.head(5)

## First look at the data

In [ ]:
df[['Close', 'Adj Close']].plot()

## Compare several securities

In [ ]:
filepath = '../data/VOO.csv'
#df2 = read_yahoo_csv(path=filepath)
df2 = retrieve_yahoo_quote(ticker='VOO', enddate=todays_date())

In [ ]:
filepath = '../data/VNQ.csv'
#df3 = read_yahoo_csv(path=filepath)
df3 = retrieve_yahoo_quote(ticker='VNQ', enddate=todays_date())

In [ ]:
plt.plot(df.index, df.Close, label='VO')
plt.plot(df2.index, df2.Close, label = 'VOO')
plt.plot(df3.index, df3.Close, label='VNQ')
plt.legend()

## Play with some basic time series functions

### Run rate

In [ ]:
df = runrate_column(df, column='Close', window=50)
df = runrate_column(df, column='Close', window=200)

In [ ]:
plt.plot(df.index, df.Close)
plt.plot(df.index, df.Close_rr50)
plt.plot(df.index, df.Close_rr200)

### Resampling

In [ ]:
df_m = resample_df(df, column='Close', resolution='M')
df_y = resample_df(df, column='Close', resolution='12M')

In [ ]:
plt.plot(df.index, df.Close)
plt.plot(df_m.index, df_m.Close['mean'])
plt.plot(df_y.index, df_y.Close['mean'])

In [ ]:
df_y.head(5)

### Volatility

In [ ]:
df = return_column(df, column='Close', window=1, shift=1)

In [ ]:
df2 = return_column(df2, column='Close', window=1, shift=1)

In [ ]:
df3 = return_column(df3, column='Close', window=1, shift=1)

In [ ]:
plt.plot(df2.index, df2.Close_sh1_sqerr)
plt.plot(df.index, df.Close_sh1_sqerr)
plt.plot(df3.index, df3.Close_sh1_sqerr)

## Differences in time

In [ ]:
startdate = '2017-01-03'
enddate   = '2017-02-09'

In [ ]:
df_plot = df.loc[(df.index >= startdate) & (df.index <= enddate), :]
plt.plot(df_plot.index, df_plot.Close)

In [ ]:
difference(df=df, column='Close', start=startdate, end=enddate)

## Security class

In [ ]:
VO = security.security('VO', start='2017-01-01', end='2017-12-31')
VOO = security.security('VOO', start='2017-01-01', end='2017-12-31')
VNQ = security.security('VNQ', start='2017-01-01', end='2017-12-31')

In [ ]:
VO.ticker, VOO.ticker, VNQ.ticker

In [ ]:
VNQ.refresh()

### Load data either from file or from Yahoo!

In [ ]:
VO.load(start='2017-01-01', end='2017-12-31')
VOO.load(start='2017-01-01', end='2017-12-31')

In [ ]:
difference(df=VO.data, column='Close', start='2017-01-01', end='2017-01-30')

In [ ]:
difference(df=VOO.data, column='Close', start='2017-01-01', end='2017-01-30')

### Name class instance

In [ ]:
VO.set_name('Vanguard Mid Cap ETF')

In [ ]:
VO.name

### Last price in data

In [ ]:
VO.get_last_price('Close')

In [ ]:
VO.last_price

### Plot outliers

In [ ]:
df_high = VNQ.data.loc[VNQ.data.Close >= VNQ.mean_price + VNQ.std_price]
df_low = VNQ.data.loc[VNQ.data.Close <= VNQ.mean_price - VNQ.std_price]

In [ ]:
plt.plot(VNQ.data.index, VNQ.data.Close)
plt.scatter(df_high.index, df_high.Close)
plt.scatter(df_low.index, df_low.Close)

### Refresh data (and save as csv)

In [ ]:
VO.refresh()

In [ ]:
VOO.refresh()

### R squared between two securities

In [ ]:
rsq(VOO, VO)

In [ ]:
rsq(VNQ, VO)

In [ ]:
rsq(VOO, VNQ)

### 50-day and 200-day rolling weighted averages for securities 

In [ ]:
def plot_trend(security=None, n=100):
    security.data = runrate_column(security.data, column='Close', window=50)
    security.data = runrate_column(security.data, column='Close', window=200)
    plt.plot(security.data.index[-n:], security.data.Close[-n:], label='1 day')
    plt.plot(security.data.index[-n:], security.data.Close_rr50[-n:], label='50 day')
    plt.plot(security.data.index[-n:], security.data.Close_rr200[-n:], label='200 day')
    plt.legend()

In [ ]:
NFLX = security.security('NFLX')

In [ ]:
SP500 = security.security('^GSPC')

In [ ]:
AMD = security.security('AMD')

In [ ]:
BIDU = security.security('BIDU')

In [ ]:
W = security.security('W')

In [ ]:
plot_trend(security=W)